<a href="https://colab.research.google.com/github/vkt1414/mhub/blob/main/totalsegmentator_mwe_vamsi_gs_util.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ModelHub - Whole Body CT Segmentation**

This notebook provides a minimal working example of TotalSegmentator, a tool for the segmentation of 104 anatomical structures from CT images. The model was trained using a wide range of imaging CT data of different pathologies from several scanners, protocols and institutions.

We test TotalSegmentator by implementing an end-to-end (cloud-based) pipeline on publicly available whole body CT scans hosted on the [Imaging Data Commons (IDC)](https://portal.imaging.datacommons.cancer.gov/), starting from raw DICOM CT data and ending with a DICOM SEG object storing the segmentation masks generated by the AI model. The testing dataset we use is external and independent from the data used in the development phase of the model (training and validation) and is composed by a wide variety of image types (from the area covered by the scan, to the presence of contrast and various types of artefacts).

The way all the operations are executed - from pulling data, to data postprocessing, and the standardisation of the results - have the goal of promoting transparency and reproducibility. Furthermore, this notebook is part of [a collection of code, notebooks and Docker containers](https://github.com/AIM-Harvard/mhub/blob/main/mhub/totalsegmentator/notebooks/totalsegmentator_mwe.ipynb) we are developing with the goal of making a wide range of machine learning models for medicine available through a standardized I/O framework.



Please cite the following article if you use this code or pre-trained models:

Wasserthal, J., Meyer, M., Breit, H.C., Cyriac, J., Yang, S. and Segeroth, M., 2022. TotalSegmentator: robust segmentation of 104 anatomical structures in CT images. arXiv preprint arXiv:2208.05868, [
https://doi.org/10.48550/arXiv.2208.05868]( 	
https://doi.org/10.48550/arXiv.2208.05868).

The original code is published on
[GitHub](https://github.com/wasserth/TotalSegmentator)  using the [Apache-2.0 license](https://github.com/wasserth/TotalSegmentator/blob/master/LICENSE).

### **Disclaimer**

The code and data of this repository are provided to promote reproducible research. They are not intended for clinical care or commercial use.

The software is provided "as is", without warranty of any kind, express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, arising from, out of or in connection with the software or the use or other dealings in the software.

# **Environment Setup**

This demo notebook is intended to be run using a GPU.

To access a free GPU on Colab:
`Edit > Notebooks Settings`.

From the dropdown menu under `Hardware accelerator`, select `GPU`. Let's check the Colab instance is indeed equipped with a GPU.

This notebook when intended to use in a workflow in Terra or CGC-Seven bridges, can also be run with out gpu

In [1]:
import os
import sys
import yaml
import time
import random
from pathlib import Path

# useful information
curr_dir   = Path().absolute()
curr_droid = !hostname
curr_pilot = !whoami

print(time.asctime(time.localtime()))
print("\nCurrent directory :{}".format( curr_dir))
print("Hostname          :", curr_droid[-1])
print("Username          :", curr_pilot[-1])
print("Python version    :", sys.version.split('\n')[0])


# ----------------------------------------

import numpy as np
import pandas as pd

print("Numpy version                : ", np.__version__)

Tue Feb 14 14:43:23 2023

Current directory :/content
Hostname          : d84991a7096f
Username          : root
Python version    : 3.8.10 (default, Nov 14 2022, 12:59:47) 
Numpy version                :  1.21.6


The authentication to Google is necessary to run BigQuery queries.

Every operation throughout the whole notebook (BigQuery, fetching data from the IDC buckets) is completely free. The only thing that is needed in order to run the notebook is the set-up of a Google Cloud project. In order for the notebook to work as intended, you will need to specify the name of the project in the cell after the authentication one.

In [2]:
# # when running on Terra or Seven bridges this cell can be commented out as 
# # we will not need any authentication to google cloud except when we want to use 
# # bigquery to download dicom files. In that case, authentication is taken care of 
# # by using a service account

# from google.colab import auth
# auth.authenticate_user()

In [3]:
## from google.colab import files 

## from google.cloud import storage
## from google.cloud import bigquery as bq

## INSERT THE ID OF YOUR PROJECT HERE!
## project_id = "idc-external-030" 

In [4]:
%%capture
# utility to make yaml files easily editable in a notebook cell
!pip install yamlmagic 

In [5]:
%load_ext yamlmagic

Throughout this Colab notebook, for image pre-processing we will use [Plastimatch](https://plastimatch.org), a reliable and open source software for image computation. We will be running Plastimatch using the simple [PyPlastimatch](https://github.com/AIM-Harvard/pyplastimatch/tree/main/pyplastimatch) python wrapper. 

In [6]:
%%capture
!apt install plastimatch 

In [7]:
#check plastimatch was correctly installed
!plastimatch --version 

plastimatch version 1.8.0


Install Apache's subversion. 

We will use subversion to clone only specific subfolders of the mhub repository.

In [8]:
%%capture
!apt install subversion 

---

Start by cloning the AIMI hub repository on the Colab instance.

The AIMI hub repository stores all the code we will use for pulling, preprocessing, processing, and postprocessing the data for this use case - as long as the others shared through AIMI hub.

In [9]:
%%capture
!svn checkout https://github.com/AIM-Harvard/mhub/trunk/mhub/mhubio mhub/mhubio
!svn checkout https://github.com/AIM-Harvard/mhub/trunk/mhub/ymldicomseg mhub/ymldicomseg
!svn checkout https://github.com/AIM-Harvard/mhub/trunk/mhub/totalsegmentator mhub/totalsegmentator

To organise the DICOM data in a more common (and human-understandable) fashion after downloading those from the buckets, we will make use of [DICOMSort](https://github.com/pieper/dicomsort). 

DICOMSort is an open source tool for custom sorting and renaming of dicom files based on their specific DICOM tags. In our case, we will exploit DICOMSort to organise the DICOM data by `PatientID` and `Modality` - so that the final directory will look like the following:

```
data/raw/nsclc-radiomics/dicom/$PatientID
 └─── CT
       ├─── $SOPInstanceUID_slice0.dcm
       ├─── $SOPInstanceUID_slice1.dcm
       ├───  ...
       │
      RTSTRUCT 
       ├─── $SOPInstanceUID_RTSTRUCT.dcm
      SEG
       └─── $SOPInstanceUID_RTSEG.dcm

```

We will also use DCMQI for converting the resulting segmentation into standard DICOM SEG objects.

In [10]:
%%capture

dcmqi_release_url = "https://github.com/QIICR/dcmqi/releases/download/v1.2.4/dcmqi-1.2.4-linux.tar.gz"
dcmqi_download_path = "/content/dcmqi-1.2.4-linux.tar.gz"
dcmqi_path = "/content/dcmqi-1.2.4-linux"

!wget -O $dcmqi_download_path $dcmqi_release_url

!tar -xvf $dcmqi_download_path

!mv $dcmqi_path/bin/* /bin

---

Let's now install example-specific python dependencies we will need.

In [11]:
%%capture

!pip install thedicomsort
!pip install pyplastimatch nnunet
!pip install TotalSegmentator

Provided everything was set up correctly, we can run the BigQuery query and get all the information we need to download the testing data from the IDC platform.

For this specific use case, we are going to be working with the "CT lymph nodes" collection hosted on IDC - which groups a collections of series that are close to whole body CT scans.

In [12]:
JSONServiceAccountFile='graceful-goods-375814-d6d8e1553699.json'
SeriesInstanceUIDs=['1.2.840.113654.2.55.219328356040179134586098264373358229860',
                    '1.3.6.1.4.1.14519.5.2.1.7009.9004.231985094753394017986102999426',
                    '1.3.6.1.4.1.14519.5.2.1.7009.9004.144965022893191407663386785886',
                    '1.3.6.1.4.1.14519.5.2.1.7009.9004.117168587881036721196334662566',
                    '1.3.6.1.4.1.14519.5.2.1.7009.9004.333778512172976907200406892074',
                    '1.3.6.1.4.1.14519.5.2.1.7009.9004.989201744992932074573140806434',
                    '1.2.840.113654.2.55.320599077732884708766859677959825659933',
                    '1.3.6.1.4.1.14519.5.2.1.7009.9004.164586013035960730340611327787',
                    '1.3.6.1.4.1.14519.5.2.1.7009.9004.106004492444389108941417962052',
                    '1.3.6.1.4.1.14519.5.2.1.7009.9004.331102856465925790130567116422'
]
FastModeStatus='True'

In [13]:
##authenticating google cloud with a service account to run bigquery
#when the new IDC storage schema is live, there will be no need to use service account

from google.cloud import storage
from google.cloud import bigquery as bq
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
     '{}/graceful-goods-375814-d6d8e1553699.json'.format(curr_dir), scopes=["https://www.googleapis.com/auth/cloud-platform"],)
bq_client = bq.Client(credentials=credentials, project=credentials.project_id)


In [14]:
# %%bigquery cohort_df --project=$project_id 

# SELECT
#   dicom_pivot_v11.PatientID,
#   dicom_pivot_v11.collection_id,
#   dicom_pivot_v11.source_DOI,
#   dicom_pivot_v11.StudyInstanceUID,
#   dicom_pivot_v11.SeriesInstanceUID,
#   dicom_pivot_v11.SOPInstanceUID,
#   dicom_pivot_v11.Modality,
#   dicom_pivot_v11.gcs_url
# FROM
#   `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
# WHERE
#   StudyInstanceUID IN (
#     SELECT
#       StudyInstanceUID
#     FROM
#       `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
#     WHERE
#       (
#         dicom_pivot_v11.collection_id IN ('Community', 'nsclc_radiomics')
#       )
#     GROUP BY
#       StudyInstanceUID
#   )
# GROUP BY
#   dicom_pivot_v11.PatientID,
#   dicom_pivot_v11.collection_id,
#   dicom_pivot_v11.source_DOI,
#   dicom_pivot_v11.StudyInstanceUID,
#   dicom_pivot_v11.SeriesInstanceUID,
#   dicom_pivot_v11.SOPInstanceUID,
#   dicom_pivot_v11.Modality,
#   dicom_pivot_v11.gcs_url
# ORDER BY
#   dicom_pivot_v11.PatientID ASC,
#   dicom_pivot_v11.collection_id ASC,
#   dicom_pivot_v11.source_DOI ASC,
#   dicom_pivot_v11.StudyInstanceUID ASC,
#   dicom_pivot_v11.SeriesInstanceUID ASC,
#   dicom_pivot_v11.SOPInstanceUID ASC,
#   dicom_pivot_v11.Modality ASC,
#   dicom_pivot_v11.gcs_url ASC

In [15]:

selection_query = """
select distinct
nlst.SeriesInstanceUID,
idc.gcs_url,
CONCAT("cp ",REPLACE(idc.gcs_url, "gs://", "s3://"), " .") as s5cmd_cp_url

 from `graceful-goods-375814.terra.nlst` nlst

 join `bigquery-public-data.idc_current.dicom_all` idc

 on nlst.SeriesInstanceUID = idc.SeriesInstanceUID

"""

cohort_df = bq_client.query(selection_query).to_dataframe()
#cohort_df=cohort_df[cohort_df["SeriesInstanceUID"]==SeriesInstanceUID]

In [16]:
# # N.B. - this works as intended only if the BQ query parses data from a single dataset
# # if not, feel free to set the name manually!
# dataset_name = cohort_df["collection_id"].values[0]
# dataset_name

In [17]:
# create the directory tree
!mkdir -p data

!mkdir -p data/idc_data
!mkdir -p data/input_data 
!mkdir -p data/output_data 

# **Parsing Cohort Information from BigQuery Tables**

We can check the various fields of the table we populated by running the BigQuery query.

This table will store one entry for each DICOM file in the dataset (therefore, expect thousands of rows!)

In [18]:
# pat_id_list = sorted(list(set(cohort_df["PatientID"].values)))

# print("Total number of unique Patient IDs:", len(pat_id_list))

# display(cohort_df.info())
# display(cohort_df.head())

# **Setup mhubio**

`mhbio` is the module of MHub that deals with all of the basic operations shared between a large majority of the models.

Let's import all of the modules we will need to run the `TotalSegmentator` pipeline from end to end.


In [19]:
!pip install pyplastimatch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
sys.path.append('.')

from mhub.mhubio.Config import Config, DataType, FileType, CT, SEG

from mhub.mhubio.modules.importer.UnsortedDicomImporter import UnsortedInstanceImporter
from mhub.mhubio.modules.importer.DataSorter import DataSorter
from mhub.mhubio.modules.convert.NiftiConverter import NiftiConverter
from mhub.mhubio.modules.convert.DsegConverter import DsegConverter
from mhub.mhubio.modules.organizer.DataOrganizer import DataOrganizer

from mhub.totalsegmentator.utils.TotalSegmentatorRunner import TotalSegmentatorRunner

For instance, the workflow for this example looks like the following:

```
DICOM CT >> NIfTI >> TotalSegmentator >> NIfTI >> DICOM SEG 
```

We want to start from DICOM CT data and save the results in DICOM SEG format. We are therefore going to need the relevant `DataType`(s) and `FileType`(s) imported (`CT` and the `SEG`) from our `Config` module. 

We will also need to import all the converters to run the aforementioned operations (`UnsortedInstanceImporter`, `DataSorter`, `NiftiConverter`, `DsegConverter`, `DataOrganizer`) and the model runner (`TotalSegmentatorRunner`).

For more in-depth explanation of the modules, see the following sections.


# **Running the Analysis for a Single Patient**

The following cells run all the processing pipeline, from pre-processing to post-processing.

Let's start by slicing the datafram storing the metadata we pulled from IDC to get a single series from a single patient.

In [21]:
# # randomly select one patient from the cohort
# pat_id = random.choice(cohort_df["PatientID"].values)
# patient_df = cohort_df[cohort_df["PatientID"] == pat_id].reset_index(drop = True)

# # select only data for which the modality is CT 
# #patient_df = patient_df[patient_df["Modality"] == "CT"].reset_index(drop = True)

# # if more than one series are available for the selected patient, pick one
# if len(np.unique(patient_df["SeriesInstanceUID"].values)) > 1:
#   series_uid = random.choice(patient_df["SeriesInstanceUID"].values)
#   patient_df = patient_df[patient_df["SeriesInstanceUID"] == series_uid].reset_index(drop = True)

# # sanity check
# assert len(np.unique(patient_df["SeriesInstanceUID"].values)) == 1

# display(patient_df.info())
# display(patient_df.head())

Write a custom configuration file containing the specifics for all the MHub modules we're going to use, using the `%%writefile` magik (from the `yamlmagic` package).

This file is going to be tailored to this specific use case and example.

The config file stores all the information needed by the different modules, such as the path to a given file or folder, to execute the end-to-end pipeline.

For instance, we specify `data_base_dir` as a `general` argument all the modules are going to share. Then, we specify the module-specific arguments.

For instance, we specify the directory where the DICOM data to be sorted are for the `UnsortedInstanceImporter`, by adding `input_dir` to the config file. We can also specify the structure of the sorted directory by setting the `structure` argument for the `DataSorter`, or which `TotalSegmentator` model to run.

In [22]:
curr_dir

PosixPath('/content')

In [23]:
%%writefile totalsegmentator_config.yml
general:
  data_base_dir: data/
modules:
  UnsortedInstanceImporter:
    input_dir: idc_data
  DataSorter:
    base_dir: data/sorted/
    structure: '%SeriesInstanceUID/dicom/%SOPInstanceUID.dcm'
  DsegConverter:
    #dicomseg_json_path: mhub/totalsegmentator/config/dicomseg_metadata_whole.json
    dicomseg_yml_path: mhub/totalsegmentator/config/dseg.yml
    skip_empty_slices: True
  TotalSegmentatorRunner:
    use_fast_mode: FastModeStatus

Writing totalsegmentator_config.yml


After defining a config file for our use case, we can initialize a `Config` object using such `yml` file.

The `Config` object is passed along to all the modules, and keeps track of all of the information that need to shared among these modules.

In [24]:
# config
config = Config('totalsegmentator_config.yml')
config.verbose = True

In the next cells, we define a utility function to pull data from the Imaging Data Commons - starting from the DataFrame `patient_df` we have previously defined - and then cross-load the data from the IDC Buckets to this Colab instance.

In [25]:
# def download_patient_data(download_path, patient_df):

#   """
#   Download raw DICOM data and run dicomsort to standardise the input format.
#   Arguments:
#     download_path : required - path to the folder where the raw data will be downloaded.
#     patient_df    : required - Pandas dataframe storing all the information required
#                                to pull data  from the IDC buckets.
#   """

#   gs_file_path = "gcs_paths.txt"
#   patient_df["gcs_url"].to_csv(gs_file_path, header = False, index = False)

#   pat_id = patient_df["PatientID"].values[0]
#   download_path = os.path.join(download_path, pat_id)

#   if not os.path.exists(download_path): os.mkdir(download_path)

#   start_time = time.time()
#   print("Copying files from IDC buckets to %s..."%(download_path))

#   !cat $gs_file_path | gsutil -q -m cp -Ir $download_path >> /dev/null

#   elapsed = time.time() - start_time
#   print("Done in %g seconds."%elapsed)

In [26]:
def download_patient_data(download_path,series_id):

  """
  Download raw DICOM data and run dicomsort to standardise the input format.
  Arguments:
    download_path : required - path to the folder where the raw data will be downloaded.
    patient_df    : required - Pandas dataframe storing all the information required
                               to pull data  from the IDC buckets.
  """

  gs_file_path = "gcs_paths.txt"
  cohort_df = bq_client.query(selection_query).to_dataframe()
  cohort_df=cohort_df[cohort_df['SeriesInstanceUID']==series_id]
  cohort_df["gcs_url"].to_csv(gs_file_path, header = False, index = False)

  download_path = os.path.join(download_path)

  if not os.path.exists(download_path): os.mkdir(download_path)

  start_time = time.time()
  print("Copying files from IDC buckets to %s..."%(download_path))

  !cat $gs_file_path | gsutil -q -m cp -Ir $download_path >> /dev/null

  elapsed = time.time() - start_time
  print("Done in %g seconds."%elapsed)

In [ ]:
def download_patient_data(download_path,series_id):

  """
  Download raw DICOM data and run dicomsort to standardise the input format.
  Arguments:
    download_path : required - path to the folder where the raw data will be downloaded.
    patient_df    : required - Pandas dataframe storing all the information required
                               to pull data  from the IDC buckets.
  """

  gs_file_path = "s5cmd_manifest.txt"
  cohort_df = bq_client.query(selection_query).to_dataframe()
  cohort_df=cohort_df[cohort_df['SeriesInstanceUID']==series_id]
  cohort_df["s5cmd_cp_url"].to_csv(gs_file_path, header = False, index = False)

  download_path = os.path.join(download_path)

  if not os.path.exists(download_path): os.mkdir(download_path)

  start_time = time.time()
  print("Copying files from IDC buckets to %s..."%(download_path))

  #!cat $gs_file_path | gsutil -q -m cp -Ir $download_path >> /dev/null
  s5cmd --endpoint-url https://storage.googleapis.com run s5cmd_manifest.txt

  elapsed = time.time() - start_time
  print("Done in %g seconds."%elapsed)

In [27]:
# # cross-load data
# for series_id in SeriesInstanceUIDs:
#   download_patient_data("data/idc_data",series_id)

We then import the DICOM data found at `data/idc_data`, as specified in the config file at:

```
general:
  data_base_dir: /content/data
modules:
  UnsortedInstanceImporter:
    input_dir: idc_data
```

In [28]:
# import a collection of unsorted DICOM data
for series_id in SeriesInstanceUIDs:
  download_patient_data("data/idc_data",series_id)

  importer = UnsortedInstanceImporter(config)
  importer.execute()
  DataSorter(config).execute()
  NiftiConverter(config).execute()
  TotalSegmentatorRunner(config).execute()
  DsegConverter(config).execute()
  # organizer = DataOrganizer(config, set_file_permissions = sys.platform.startswith('linux'))
  # organizer.setTarget(DataType(FileType.NIFTI, CT), "{}/data/output_data/[i:SeriesID]/[path]".format(curr_dir))
  # organizer.setTarget(DataType(FileType.DICOMSEG, SEG), "{}/data/output_data/[i:SeriesID]/TotalSegmentator.seg.dcm".format(curr_dir))
  # organizer.execute()
  archive_fn = "{}.zip".format(series_id)

  # import pydicom
  # seg_series = pydicom.dcm_read(f"output_data/{series_id}/TotalSegmentator.dcm")
  # series_uid = seg_series.SeriesInstanceUID

  # # cp TotalSegmentator.seg.dcm to {series_uid}.dcm and zip

  try:
     os.remove(archive_fn)
  except OSError:
     pass
  !zip -j -r "$archive_fn" "data/sorted/$series_id/seg.dcm" 
  #!find data/idc_data -mindepth 1 -delete
  #!rm -r data/sorted gcs_paths.txt data/output_data/
  !rm -r data/ /app gcs_paths.txt 
  !mkdir -p data/
  !mkdir -p data/idc_data
  !mkdir -p data/input_data 
  !mkdir -p data/output_data 
#/content/data/sorted/1.2.840.113654.2.55.219328356040179134586098264373358229860/image.nii.gz

Copying files from IDC buckets to data/idc_data...
Done in 63.6645 seconds.

--------------------------
Start UnsortedInstanceImporter
Done in 0.000256062 seconds.

--------------------------
Start DataSorter
sorting schema: data/sorted//%SeriesInstanceUID/dicom/%SOPInstanceUID.dcm
>> run:  dicomsort -k -u data/idc_data data/sorted//%SeriesInstanceUID/dicom/%SOPInstanceUID.dcm
adding ct in dicom format with resolved path:  data/sorted/1.2.840.113654.2.55.219328356040179134586098264373358229860/dicom
Done in 0.929038 seconds.

--------------------------
Start NiftiConverter

Running 'plastimatch convert' with the specified arguments:
  --input data/sorted/1.2.840.113654.2.55.219328356040179134586098264373358229860/dicom
  --output-img data/sorted/1.2.840.113654.2.55.219328356040179134586098264373358229860/image.nii.gz
... Done.
Done in 35.2712 seconds.

--------------------------
Start TotalSegmentatorRunner
Running TotalSegmentator in fast mode ('--fast', 3mm): 
>> run ts:  TotalSegmen

In [29]:
# archive_fn = "{}.zip".format(series_id)
# try:
#   os.remove(archive_fn)
# except OSError:
#   pass
# !zip -j -r $archive_fn "{}/data/output_data".format(curr_dir) "{}/data/input_data".format(curr_dir)
# !find data/idc_data -mindepth 1 -delete
# !rm -r data/sorted gcs_paths.txt data/

In [30]:
# %%capture
  
# for series_id in SeriesInstanceUIDs:
#   archive_fn = "%s.zip"%(series_id)
#   try:
#     os.remove(archive_fn)
#   except OSError:
#     pass
#   !zip -j -r $archive_fn "{}/data/output_data".format(curr_dir) "{}/data/input_data".format(curr_dir)

After importing the data, we sort the DICOM files in the fashion specified in the config file - and after, we convert to NIfTI, which is the format accepted by TotalSegmentator.

In [31]:
# # sort such collection of DICOM data using dicomsort
# for series_id in SeriesInstanceUIDs:
#   DataSorter(config).execute()

In [32]:
# # convert the DICOM data to NIfTI, as required by TotalSegmentator, using plastimatch
# for series_id in SeriesInstanceUIDs:
#   NiftiConverter(config).execute()

Finally, we run TotalSegmentator using the parameters specified at the `TotalSegmentatorRunner` module of the config file, and convert the results back to DICOM SEG using the parameters set in the `DsegConverter` (`dicomseg_yml_path` and `dicomseg_json_path`).

In [33]:
# # run the inference phase 
# for series_id in SeriesInstanceUIDs:
#   TotalSegmentatorRunner(config).execute()

In [34]:
# # convert the results to DICOM SEG
# DsegConverter(config).execute()

As a last step, we run MHub's `DataOrganizer` to organize the output data in a predetermined and standardized structure.

In [35]:
# # organize data into output folder
# # FIXME: don't save stuff under /app
# organizer = DataOrganizer(config, set_file_permissions = sys.platform.startswith('linux'))
# organizer.setTarget(DataType(FileType.NIFTI, CT), "{}/data/output_data/[i:SeriesID]/[path]".format(curr_dir))
# organizer.setTarget(DataType(FileType.DICOMSEG, SEG), "{}/data/output_data/[i:SeriesID]/TotalSegmentator.seg.dcm".format(curr_dir))
# organizer.execute()

---

# **Downloading the Results Locally**

After the end-to-end processing is done, we can download the results (and the pre-processed input data) running the following cells.

In [36]:
# %%capture
# for series_id in SeriesInstanceUIDs:
#   archive_fn = "%s.zip"%(series_id)

#   try:
#     os.remove(archive_fn)
#   except OSError:
#     pass

#   !zip -j -r $archive_fn "{}/data/output_data".format(curr_dir) "{}/data/input_data".format(curr_dir)

In [37]:
# #!find data/sorted/* -maxdepth 1 -type f -delete

# !find data/idc_data -mindepth 1 -delete
# !rm -r data/sorted gcs_paths.txt data/

In [38]:
# filesize = os.stat(archive_fn).st_size/1024e03
# print('Starting the download of "%s" (%2.1f MB)...\n'%(archive_fn, filesize))

# files.download(archive_fn)